In [1]:
from pyspark.sql import SparkSession
import timeit 

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.131:7077") \
        .appName("Team7_code.ipynb")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max",5)\
        .config("spark.worker.instances",5)\
        .config("spark.executor.cores",1)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()
#\#)\
# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")
print("Started Session")

Started Session


In [2]:
def create_spark_dataframe(filename):
    df = spark_session.read.json('hdfs://192.168.2.131:9000/user/ubuntu/{}'.format(filename))
    return df

start_time_0 = timeit.default_timer() 

#df = create_spark_dataframe('RC_2005-12.json')
df = create_spark_dataframe('RC_2011-08.json')
#df = create_spark_dataframe('RC_2009-05.json')
df.printSchema()
print("execution time {}s".format(timeit.default_timer()-start_time_0))

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)

execution time 77.44055639300495s


In [67]:
from pyspark.sql.functions import desc, col

#num_comments = df.count()
#print('number of comments: {}'.format(num_comments))

start_time_0 = timeit.default_timer() 

author_comment_counts = df.groupBy('author').count()
author_comment_counts.cache()
author_comment_counts.sort(desc("count")).show()
print("execution time {}s".format(timeit.default_timer()-start_time_0))


+------------------+-------+
|            author|  count|
+------------------+-------+
|         [deleted]|2442948|
|         mileylols|   4594|
|            Lots42|   4047|
|       GenJonesMom|   3294|
|     dispatcher_83|   3006|
|           JBgreen|   2817|
|            Spongi|   2731|
|Release_the_KRAKEN|   2680|
|          Osmodius|   2579|
|      redhatnation|   2481|
|      mavriksfan11|   2465|
|   original-finder|   2421|
|    GhostedAccount|   2406|
|         Warlizard|   2296|
|       StickDoctor|   2273|
|           Moridyn|   2053|
|          Ali-Sama|   2041|
|         edubation|   2009|
|          G_Morgan|   1974|
|       silverhydra|   1952|
+------------------+-------+
only showing top 20 rows

execution time 70.14901283899962s


In [13]:

start_time_0 = timeit.default_timer() 
#author_comment_counts.select("count").rdd.map(tuple).map(lambda x: x[0]).map(lambda x: x**2).collect()
df_tmp = df.select("score").rdd.map(tuple).map(lambda x: x[0]).cache()
print(df_tmp.map(lambda x: x**2).reduce(lambda x,y: x+y))

print("execution time {}s".format(timeit.default_timer()-start_time_0))

start_time_0 = timeit.default_timer() 
#author_comment_counts.select("count").rdd.map(tuple).map(lambda x: x[0]).map(lambda x: x**2).collect()
df_tmp.map(lambda x: x**2).reduce(lambda x,y: x+y)

print("execution time {}s".format(timeit.default_timer()-start_time_0))

7964577335
execution time 74.15956350398483s
execution time 5.6745331730053294s


In [32]:
start_time_0 = timeit.default_timer() 
author_score = df.groupBy('author').sum('score')
author_score.sort(desc("sum(score)")).show()
print("execution time {}s".format(timeit.default_timer()-start_time_0))

+-------------+----------+
|       author|sum(score)|
+-------------+----------+
|   paulgraham|        98|
|     mattknox|        77|
|    [deleted]|        58|
|      bugbear|        43|
| michaelneale|        39|
|     dstowell|        38|
|         spez|        36|
|      AaronSw|        30|
|     JimThome|        29|
|        sempf|        29|
|    bolinfest|        28|
|       davidw|        27|
|          Zak|        26|
|     symbiont|        25|
|     binladen|        24|
|     enjahova|        23|
|         cg84|        23|
|brendankohler|        21|
|       dylanm|        20|
|     beastboy|        19|
+-------------+----------+
only showing top 20 rows

execution time 2.5746123989956686s


In [40]:
start_time_0 = timeit.default_timer() 

author_df = author_comment_counts.join(author_score, author_comment_counts.author == author_score.author).withColumnRenamed("sum(score)", "score")
author_df.withColumn('Average Score', author_df['score']/author_df['count']).sort(desc('Average Score')).show()
print("execution time {}s".format(timeit.default_timer()-start_time_0))

root
 |-- author: string (nullable = true)
 |-- count: long (nullable = false)
 |-- author: string (nullable = true)
 |-- score: long (nullable = true)

+--------------+-----+--------------+-----+-----------------+
|        author|count|        author|score|    Average Score|
+--------------+-----+--------------+-----+-----------------+
|          Grue|    1|          Grue|   16|             16.0|
|TenebraeVision|    1|TenebraeVision|   16|             16.0|
|        fergie|    1|        fergie|   12|             12.0|
|        bstard|    1|        bstard|   11|             11.0|
|         sempf|    3|         sempf|   29|9.666666666666666|
|     bolinfest|    4|     bolinfest|   28|              7.0|
|    HiggsBoson|    1|    HiggsBoson|    7|              7.0|
|      kanagawa|    1|      kanagawa|    7|              7.0|
|    paulgraham|   15|    paulgraham|   98|6.533333333333333|
|         arkas|    2|         arkas|   13|              6.5|
|      dstowell|    6|      dstowell|   3

In [3]:
import re
def pre_process_text(text):
    text = text.lower()
    text = text.replace('\r', '')
    text = text.replace('\n', '')
    text = text.strip()
    text=text.split(' ')
    text = map(lambda x: re.sub(r'[^a-zA-Z ]+', '', x), text)
    return text

start_time_0 = timeit.default_timer() 

df_body = df.select("body").rdd.map(tuple).cache()

print(df_body.map(lambda x: x[0]).flatMap(pre_process_text).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y).sortBy(lambda x: x[1], ascending=False).take(10))

print("execution time {}s".format(timeit.default_timer()-start_time_0))

[('the', 13768265), ('to', 9377579), ('a', 8884361), ('', 8400419), ('i', 8302200), ('and', 7374302), ('of', 6268364), ('you', 5644058), ('that', 5097987), ('it', 5023729)]
execution time 361.68467402498936s


In [41]:
subreddits = df.select('subreddit').dropDuplicates().rdd.map(tuple).map(lambda x: x[0]).collect()

df.rdd.map(tuple)
subreddit = subreddits[0]

for 


['MensRights', 'anime', 'travel', 'Amateur', 'Entrepreneurship', 'coding_tutorials', 'Kneejerk', 'left4dead', 'occult', 'scifi', 'girlskissing', 'conspiracyhub', 'craigslist', 'television', 'GenderTheory', 'pharmacology', 'hockey', 'australia', 'Sexy', 'directors', 'chemistry', 'religiouscirc', 'lies', 'portugal', 'CampingandHiking', 'mjstrains', 'Gourmet', 'Antitheism', 'Winnipeg', 'hipsters', 'hackintosh', 'ledelse', 'battery', 'PS3', 'evosoc', 'redditchan', 'Boxing', 'YooouuuTuuube', 'MarijuanaNICEASSNUGS', 'stocks', 'IndustrialDesign', 'Marijuana', 'space', 'ZenHabits', 'iphone', 'SOS', 'spotifylists', 'Documentaries', 'astro', 'DelawarePolitics', 'designthought', 'financial', 'microsoft', 'tinfoilhat', 'joos', 'crime', 'environment2', 'RedditRomania', 'HoangVantu', 'videos', 'ideasfortheadmins', 'AskReddit', 'productivity', 'Buddha', 'gotya', '2012', 'latinascaliente', 'Igor', 'debian', 'stonerrock', 'testreddittryitfornj', 'deepthroat', 'advertising', 'futurama', 'Webrecsol', 'Ne

In [47]:
# release the cores for another application!
spark_context.stop()
print("Ended Session")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38677)
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/context.py:460: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  warnings.warn(


Ended Session
